In [40]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten,BatchNormalization
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.callbacks import EarlyStopping

# import warnings
# warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from PIL import Image
import os
from scipy import misc
from  collections import Counter


In [2]:
PATH_TO_IMAGES="/home/roman/DS_part/image_10_array/"
# PATH_TO_LABELS="/Volumes/HDD/Datasets/Курсач/vgg_face_dataset/files/_all_in_one_10.csv"

### Работа с данными

In [3]:
#Загрузка данных
X=np.load(PATH_TO_IMAGES+"X_data.npy")
y=np.load(PATH_TO_IMAGES+"y_data.npy")

In [4]:
#Кодирую имена числами
la=LabelEncoder()
y_copy=y.copy()
y=la.fit_transform(y)

In [5]:
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=17,stratify=y)

In [6]:
num_train, height, width,depth = X_train.shape 
num_test = X_test.shape[0]
num_classes = np.unique(y_train).shape[0] 

#new
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_train) # Normalise data to [0, 1] range


Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

### Создание архитектуры и обучение модели

In [38]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 30 # we iterate 200 times over the entire training set
kernel_size = (3,3) # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

#### Без батч нормализации

In [24]:
inp = Input(shape=(height, width,depth))
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Conv2D(conv_depth_1, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Conv2D(conv_depth_1, kernel_size,  padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(drop_1)
conv_4 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=2, validation_split=0.1) # ...holding out 10% of the data for validation

#### Батч нормализация до relu

In [27]:
inp = Input(shape=(height, width,depth))
# batch_norm -> Conv [32] -> batch_norm -> Conv [32] -> Pool (with dropout on the pooling layer)
batch_norm_1=BatchNormalization()(inp)
conv_1 = Conv2D(conv_depth_1, kernel_size, padding='same', activation='relu')(batch_norm_1)
batch_norm_2=BatchNormalization()(conv_1)
conv_2 = Conv2D(conv_depth_1, kernel_size,  padding='same', activation='relu')(batch_norm_2)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# batch_norm -> Conv [64] -> batch_norm -> Conv [64] -> Pool (with dropout on the pooling layer)
batch_norm_3=BatchNormalization()(drop_1)
conv_3 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(batch_norm_3)
batch_norm_4=BatchNormalization()(conv_3)
conv_4 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(batch_norm_4)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D,batch_norm -> apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
batch_norm_5=BatchNormalization()(flat)
hidden = Dense(hidden_size, activation='relu')(batch_norm_5)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=2, validation_split=0.1) # ...holding out 10% of the data for validation

#### Батч нормализация после relu

In [35]:
inp = Input(shape=(height, width,depth))
# Conv [32] -> batch_norm -> Conv [32] -> batch_norm -> Pool (with dropout on the pooling layer)
conv_1 = Conv2D(conv_depth_1, kernel_size, padding='same', activation='relu')(inp)
batch_norm_1=BatchNormalization()(conv_1)
conv_2 = Conv2D(conv_depth_1, kernel_size,  padding='same', activation='relu')(batch_norm_1)
batch_norm_2=BatchNormalization()(conv_2)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(batch_norm_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> batch_norm -> Conv [64] -> batch_norm -> Pool (with dropout on the pooling layer)
conv_3 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(drop_1)
batch_norm_3=BatchNormalization()(conv_3)
conv_4 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(batch_norm_3)
batch_norm_4=BatchNormalization()(conv_4)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(batch_norm_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU -> batch_norm (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
batch_norm_5=BatchNormalization()(hidden)
drop_3 = Dropout(drop_prob_2)(batch_norm_5)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=2, validation_split=0.1) # ...holding out 10% of the data for validation

#### С ранней остоновой и батч норма после relu

In [44]:
inp = Input(shape=(height, width,depth))
# Conv [32] -> batch_norm -> Conv [32] -> batch_norm -> Pool (with dropout on the pooling layer)
conv_1 = Conv2D(conv_depth_1, kernel_size, padding='same', activation='relu')(inp)
batch_norm_1=BatchNormalization()(conv_1)
conv_2 = Conv2D(conv_depth_1, kernel_size,  padding='same', activation='relu')(batch_norm_1)
batch_norm_2=BatchNormalization()(conv_2)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(batch_norm_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> batch_norm -> Conv [64] -> batch_norm -> Pool (with dropout on the pooling layer)
conv_3 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(drop_1)
batch_norm_3=BatchNormalization()(conv_3)
conv_4 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(batch_norm_3)
batch_norm_4=BatchNormalization()(conv_4)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(batch_norm_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
## Now flatten to 1D, apply FC -> ReLU -> batch_norm (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
batch_norm_5=BatchNormalization()(hidden)
drop_3 = Dropout(drop_prob_2)(batch_norm_5)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

earlyStopping=EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')

model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=2, validation_split=0.1,callbacks=[earlyStopping]) # ...holding out 10% of the data for validation

Train on 10076 samples, validate on 1120 samples
Epoch 1/30
 - 72s - loss: 7.7614 - acc: 0.0036 - val_loss: 7.4890 - val_acc: 0.0071
Epoch 2/30
 - 68s - loss: 6.7639 - acc: 0.0429 - val_loss: 6.6443 - val_acc: 0.0473
Epoch 3/30
 - 68s - loss: 5.0724 - acc: 0.1785 - val_loss: 5.7611 - val_acc: 0.1277
Epoch 4/30
 - 68s - loss: 3.3221 - acc: 0.4067 - val_loss: 5.2456 - val_acc: 0.2089
Epoch 5/30
 - 68s - loss: 1.7177 - acc: 0.6858 - val_loss: 4.4773 - val_acc: 0.3107
Epoch 6/30
 - 68s - loss: 0.6877 - acc: 0.8959 - val_loss: 4.4561 - val_acc: 0.3187
Epoch 7/30
 - 68s - loss: 0.3475 - acc: 0.9536 - val_loss: 4.3798 - val_acc: 0.3259
Epoch 8/30
 - 68s - loss: 0.1824 - acc: 0.9813 - val_loss: 4.4637 - val_acc: 0.3277


### Проверка качества модели

In [45]:
# 30 (with batch_norm after relu and early stopping)
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

4799/4799 [==============================] - 10s 2ms/step


[16.099583043334732, 0.0008335069806209627]

In [43]:
# 30 (with batch_norm after relu and early stopping)
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

4799/4799 [==============================] - 10s 2ms/step


[16.079677779715766, 0.001875390706397166]

In [37]:
# 10 (with batch_norm after relu)
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

4799/4799 [==============================] - 10s 2ms/step


[16.107818535154724, 0.000625130235465722]

In [29]:
# 3 (with batch_norm before relu)
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

4799/4799 [==============================] - 9s 2ms/step


[16.099204540153323, 0.00041675349031048136]

In [23]:
# 3 (with batch_norm after relu)
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

4799/4799 [==============================] - 10s 2ms/step


[16.09150168467969, 0.0014586372160866848]

In [26]:
# 3 
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

4799/4799 [==============================] - 7s 1ms/step


[7.7762248807709575, 0.00020837674515524068]

In [11]:
# 30
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

4799/4799 [==============================] - 7s 1ms/step


[7.7700840440683745, 0.00041675349031048136]

In [11]:
#200
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

4799/4799 [==============================] - 6s 1ms/step


[7.782045051986462, 0.000625130235465722]

### Вывод
Попробовать:
- Изменить архитектуру нейросети
 - Батч норм
- Попробовать больше данных (на одного пользователя а не пользователей)